In [1]:
from owlready2 import get_ontology

* Owlready2 * Warning: optimized Cython parser module 'owlready2_optimized' is not available, defaulting to slower Python implementation


In [2]:
onto = get_ontology('./fma.owl')
onto.load()

get_ontology("http://purl.org/sig/ont/fma.owl#")

In [3]:
'''
def get_nodes(onto, main_label, labels=[]):
    for main in onto.search(label = main_label):
        #import ipdb; ipdb.set_trace()
        for sub in main.subclasses():
            labels = get_members(onto, sub.label, labels)
        labels.append(main.label[0])
    return labels 

def get_subclasses(onto, main_label, subclasses=dict({})):
    for main in onto.search(label = main_label):
        subs = []
        for sub in main.subclasses():
            subs.append(sub.label[0])
            subclasses = get_subclasses(onto, sub.label, subclasses)
        if len(subs) > 0:
            subclasses[main.label[0]] = subs
    return subclasses 
'''

'\ndef get_nodes(onto, main_label, labels=[]):\n    for main in onto.search(label = main_label):\n        #import ipdb; ipdb.set_trace()\n        for sub in main.subclasses():\n            labels = get_members(onto, sub.label, labels)\n        labels.append(main.label[0])\n    return labels \n\ndef get_subclasses(onto, main_label, subclasses=dict({})):\n    for main in onto.search(label = main_label):\n        subs = []\n        for sub in main.subclasses():\n            subs.append(sub.label[0])\n            subclasses = get_subclasses(onto, sub.label, subclasses)\n        if len(subs) > 0:\n            subclasses[main.label[0]] = subs\n    return subclasses \n'

In [37]:
def get_nodes(onto, main_label, labels=[]):
    for main in onto.search(label = main_label):
        subs = list(main.subclasses())
        while subs:
            subc = subs.pop(0)
            labels.append(subc.label[0])
            subs = subs + list(subc.subclasses())
    labels = list(dict.fromkeys(labels))
    return labels

def get_subclasses(onto, main_label, apply=None):
    subclasses = dict()
    for main in onto.search(label = main_label):
        subs = [main]
        while subs:
            subc = subs.pop(0)
            temp = list(subc.subclasses())
            if len(temp) > 0:
                subclasses[subc.label[0]] = [e.label[0] for e in temp]
                subs = subs + temp
    if apply is not None:
        subclasses = apply(subclasses)
    return subclasses

def parse_isSub(subc):
    subclass = []
    for k, v in subc.items():
        for elem in v:
            subclass.append((k, elem))
    return subclass

def get_synonyms(onto, main_label, apply=None):
    synonyms = dict()
    for main in onto.search(label = main_label):
        subs = [main]
        while subs:
            subc = subs.pop(0)
            temp = list(subc.subclasses())
            syn = subc.synonym
            if len(syn) > 0:
                synonyms[subc.label[0]] = syn
            subs = subs + temp
    if apply is not None:
        synonyms = apply(synonyms)
    return synonyms

def parse_isSyn(syn):
    syns = []
    for k, v in syn.items():
        for elem in v:
            syns.append((k, elem))
            syns.append((elem, k))
    return syns

In [38]:
#nodes = get_nodes(onto, 'Anatomical entity')
#subc = get_subclasses(onto, 'Anatomical entity', apply=parse_isSub)
#syn = get_synonyms(onto, 'Anatomical entity', apply=parse_isSyn)

In [39]:
from neurolang import frontend as fe
dl = fe.NeurolangDL()

nodes = get_nodes(onto, 'Anatomical entity')
subc = get_subclasses(onto, 'Anatomical entity', apply=parse_isSub)
syn = get_synonyms(onto, 'Anatomical entity', apply=parse_isSyn)

dl.add_tuple_set(
    ((str(e),) for e in nodes),
    name='nodes'
);

dl.add_tuple_set(
    ((str(e1), str(e2),) for e1, e2 in subc),
    name='subclasses'
);

dl.add_tuple_set(
    ((str(e1), str(e2),) for e1, e2 in syn),
    name='synonyms'
);

In [ ]:
x = dl.new_symbol(name='x')
y = dl.new_symbol(name='y')
val = dl.new_symbol(name='val')
isSub = dl.new_symbol(name='isSub')
isSyn = dl.new_symbol(name='isSyn')

#query1 = dl.query(val(x), dl.symbols.nodes(x))
query2 = dl.query(isSyn(val(x), val(y)), dl.symbols.synonyms(x, y))
query3 = dl.query(isSub(val(x),val(y)), dl.symbols.subclasses(x, y))

In [15]:
query = dl.query(val(y), isSub(val('Brain'), val(y)))

StopIteration: 

In [40]:
nodes.sort()

In [43]:
subc.sort()

In [44]:
subc

[('A1 pulley of index finger', 'A1 pulley of left index finger'),
 ('A1 pulley of index finger', 'A1 pulley of right index finger'),
 ('A1 pulley of little finger', 'A1 pulley of left little finger'),
 ('A1 pulley of little finger', 'A1 pulley of right little finger'),
 ('A1 pulley of middle finger', 'A1 pulley of left middle finger'),
 ('A1 pulley of middle finger', 'A1 pulley of right middle finger'),
 ('A1 pulley of ring finger', 'A1 pulley of left ring finger'),
 ('A1 pulley of ring finger', 'A1 pulley of right ring finger'),
 ('A2 pulley of index finger', 'A2 pulley of left index finger'),
 ('A2 pulley of index finger', 'A2 pulley of right index finger'),
 ('A2 pulley of little finger', 'A2 pulley of left little finger'),
 ('A2 pulley of little finger', 'A2 pulley of right little finger'),
 ('A2 pulley of middle finger', 'A2 pulley of left middle finger'),
 ('A2 pulley of middle finger', 'A2 pulley of right middle finger'),
 ('A2 pulley of ring finger', 'A2 pulley of left ring fin

## Test zone

In [93]:
trp = onto.get_triples()[0]

In [99]:
trp

(302, 6, 80)

In [100]:
print(list(onto.properties())[6], list(onto.classes())[302])

fma.days_post-fertilization fma.fma0323595


In [21]:
onto.get_triples()[:3]

[(302, 6, 80), (302, 308, 307), (309, 6, 15)]

In [30]:
list(onto.properties())[:3]

[fma.AP_position, fma.Latin_name__TA_, fma.action_type]

In [48]:
list(onto.object_properties())[:3]

[fma.adheres_to, fma.connected_to, fma.adjacent_to]

In [49]:
list(onto.data_properties())[:3]

[fma.AP_position, fma.Latin_name__TA_, fma.action_type]

In [50]:
list(onto.classes())[-3:]

[fma.fma9994, fma.fma9998, fma.fma9999]

In [47]:
list(onto.individuals())[:4]

[fma.David_Clunie, fma.Robert_Baud]

In [135]:
lst = list(onto.search(label = "Segment of forebrain")[0].subclasses())

In [136]:
lst[0].label

['Segment of telencephalon']

In [115]:
cl = onto.search(label = "Cerebral hemisphere lobe")

In [111]:
cl.FMAID

[61823]

In [112]:
a = onto.search(FMAID = 61823)[0].synonym

In [113]:
a

['Cerebral hemisphere lobe', 'Cerebral lobe']

In [53]:
graph = default_world.as_rdflib_graph()

In [69]:
import pandas as pd
dp = pd.DataFrame(graph.predicates()).drop_duplicates()

In [70]:
do = pd.DataFrame(graph.objects()).drop_duplicates()

In [81]:
graph.identifier()

TypeError: 'BNode' object is not callable

In [ ]:
res